<a href="https://www.kaggle.com/code/averma111/pytorch-rnn-simple?scriptVersionId=129450774" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import torch
import numpy as np


In [8]:
text = ['hey how are you','good i am fine','have a nice day']
chars = set(''.join(text))
int2chars = dict(enumerate(chars))
char2ints = {char: ind for ind, char in int2chars.items()}

In [10]:
maxlen = len(max(text, key=len))

for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

In [12]:
input_seq = []
target_seq = []
for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: have a nice da
Target Sequence: ave a nice day


In [13]:
for i in range(len(text)):
    input_seq[i] = [char2ints[character] for character in input_seq[i]]
    target_seq[i] = [char2ints[character] for character in target_seq[i]]

In [15]:
dict_size = len(char2ints)
seq_len = maxlen - 1
batch_size = len(text)

In [16]:
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [17]:
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [18]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [19]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [21]:
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = torch.nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = torch.nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [24]:
# Instantiate the model with hyperparameters
model = RNNModel(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [25]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3822
Epoch: 20/100............. Loss: 2.0276
Epoch: 30/100............. Loss: 1.6269
Epoch: 40/100............. Loss: 1.2749
Epoch: 50/100............. Loss: 0.9537
Epoch: 60/100............. Loss: 0.6746
Epoch: 70/100............. Loss: 0.4613
Epoch: 80/100............. Loss: 0.3155
Epoch: 90/100............. Loss: 0.2249
Epoch: 100/100............. Loss: 0.1702


In [29]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2ints[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = torch.nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2chars[char_ind], hidden

In [30]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [31]:
sample(model, 15, 'good')

'good i am fine '

In [32]:
sample(model, 15, 'day')

'day fine a nice'

In [33]:
sample(model, 15, 'hey')

'hey how are you'